<a href="https://colab.research.google.com/github/adithya398/2DPathFinder/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import imageio.v3 as imageio
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

# convert image to binary
def to_binary(image):
    if image.ndim == 3:  # RGB image
        image = image[..., 0]
    binary = (image < 128).astype(np.uint8)  # black = 1, white = 0
    return binary

# check exist path between 2 point
def path_exists(universe, start_x, start_y, end_x, end_y):
    h, w = universe.shape
    visited = np.zeros_like(universe, dtype=bool)
    queue = deque([(start_x, start_y)])
    visited[start_y, start_x] = True

    while queue:
        x, y = queue.popleft()
        if (x, y) == (end_x, end_y):
            return True
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < w and 0 <= ny < h and universe[ny, nx] == 1 and not visited[ny, nx]:
                visited[ny, nx] = True
                queue.append((nx, ny))
    return False

# Extraction and Returning actual path
def find_path(universe, start_x, start_y, end_x, end_y):
    h, w = universe.shape
    visited = np.zeros_like(universe, dtype=bool)
    prev = dict()
    queue = deque([(start_x, start_y)])
    visited[start_y, start_x] = True

    while queue:
        x, y = queue.popleft()
        if (x, y) == (end_x, end_y):
            # Reconstruct path
            path = [(x, y)]
            while (x, y) in prev:
                x, y = prev[(x, y)]
                path.append((x, y))
            return path[::-1]

        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < w and 0 <= ny < h and universe[ny, nx] == 1 and not visited[ny, nx]:
                visited[ny, nx] = True
                prev[(nx, ny)] = (x, y)
                queue.append((nx, ny))
    return None

# Visualize path on the image
def visualize_path(universe, path):
    vis = np.stack([universe*255]*3, axis=-1).astype(np.uint8)
    for x, y in path:
        vis[y, x] = [255, 0, 0]  # Mark path in red
    plt.imshow(vis)
    plt.title("Path Visualization")
    plt.axis("off")
    plt.show()


from google.colab import files
uploaded = files.upload()


image = imageio.imread('universe1.png')
binary = to_binary(image)

# define start and end cordination
start = (10, 10)
end = (100, 100)

# Run pathfinding
print("Checking if path exists...")
if path_exists(binary, *start, *end):
    print("✅ Path exists!")
    path = find_path(binary, *start, *end)
    if path:
        visualize_path(binary, path)
else:
    print("❌ No path found.")
